<a href="https://colab.research.google.com/github/Rocio206/ADA-Informes/blob/main/Cutting_a_Rod_Informe_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Cutting a Rod**
##**1. Descripción del problema**

**Entrada**: Largo de varilla $n$, vector de precios por cada medida $p[1..n]$

**Salida**:  **Retorno máximo** $r_n$ que se puede obtener cortando la varilla y vendiendo las partes.

El enunciado del problema es bastante simple, dada una varilla de cierta longitud $n$ y el precio asociado con cada pieza de la varilla, la varilla debe cortarse y venderse. El proceso de corte debe ser tal que la **cantidad obtenida de la venta es máximo**. También dado, la cantidad es diferente para diferentes tamaños.

Dado que el valor buscado es un **maximo**, podemos decir que nos encontramos ante un **problema de optimización**.

####**Subestructura óptima:**
Cualquier solución óptima (aparte de la solución que no hace cortes) para una varilla de longitud $n > 2$ debe incluir el resultado de al menos un subproblema: una pieza de $n-i > 1$. Lo que es lo mismo que decir que la **solución óptima al problema original incorpora soluciones óptimas a los subproblemas, que pueden resolverse de forma independiente**.

Para cualquier longitud de varilla $n$, podemos determinar el valor maximo de venta $r_n$ tomando el máximo de:

*   $p_n$ : el precio que obtenemos al no hacer un corte
*   $r_1 + r$<sub>$n−1$</sub> : el valor maximo de venta de una varilla de largo 1 y una de $n -1$.

*   $r_2 + r$<sub>$n−2$</sub> : el valor maximo de venta de una varilla de largo 2 y una de $n -2$.
*   ...
*   $r$<sub>$n−1$</sub>$+ r_1$

[![Captura-de-pantalla-2022-10-03-203747.png](https://i.postimg.cc/wBrCfYCB/Captura-de-pantalla-2022-10-03-203747.png)]

Entonces, $r_n = max ( p_n $,  $r_1 + r$<sub>$n−1$</sub> , $r_2 + r$<sub>$n−2$</sub>  , .... $r$<sub>$n−1$</sub>$+ r_1$ )

Podemos simplificar esta ecuación dejando dos subproblemas, considerando que una solucion optima consiste en una primera pieza de longitud $i$ y una varilla de longitud $n - i$ cortada de forma optima.

 >$r_n=max(p_i+r$<sub>$n−i$</sub>$)$, donde $1≤i≤n$

##**2. Solución Recursiva**

###**2.1 Implementación**

En el código a continuación se implementa la solución recursiva para el problema de **Cutting a Rod**. 

Además de el retorno máximo posible, tambien retornar los cortes necesarios la cantidad de llamadas recursivas que realiza el algoritmo.

In [ ]:
from math import inf
import random

def CutRod(p,n,r):
  r+= 1
  c = -1
  if n == 0:
    return 0, 0, r

  q = -inf
  for i in range(1, n+1):
    m, c2, r = CutRod(p, n-i, r)

    if q < p[i-1]+ m:
      q = p[i-1]+ m
      if i <= n //2:
        c += 1
      else :
        c = n - i

  return q, c, r 

## EJEMPLO
p = [1,5,8,9,10,17,17,20,24,30]
n = 8
m, c, r = CutRod(p, n , -1)

print("Tabla de precios:",p)
print()

for i in range(1,5):
  n = random.randint(1,10)
  m, c, r = CutRod(p, n , -1)
 
  print("Maxima ganancia para una varilla de longitud", n ,"es", m,",")
  print("fueron necesarios", c,"cortes y se realizaron", r,"recurciones.")
  print()


Tabla de precios: [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

Maxima ganancia para una varilla de longitud 5 es 13 ,
fueron necesarios 1 cortes y se realizaron 31 recurciones.

Maxima ganancia para una varilla de longitud 2 es 5 ,
fueron necesarios 0 cortes y se realizaron 3 recurciones.

Maxima ganancia para una varilla de longitud 5 es 13 ,
fueron necesarios 1 cortes y se realizaron 31 recurciones.

Maxima ganancia para una varilla de longitud 7 es 18 ,
fueron necesarios 0 cortes y se realizaron 127 recurciones.



###**2.2 Descripción del Algoritmo**

El algoritmo recibe como entrada en arreglo $p$ que contiene los precios de las varillas por longitud y un entero $n$ que corresponde al largo de nuestra varilla.

Teniendo en cuenta que para una varilla de longitud $n$, como hacemos $n-1$ cortes, hay 2<sup>n-1</sup> formas de cortar la varilla.

Durante cada iteración $i$, para una varilla de longitud $n$ se divide en una parte de longitud $i$ que tiene precio $p[i]$ y se calcula el maximo valor posible del resto de la varilla de longitud $n-i$ aplicado recursivamente función.


####**Ejemplo**

Supongamos que tenemos un problema de tamaño 4:

>2<sup>n-1</sup> = 2<sup>3</sup> = 8 formas de cortar la varilla.


[![Captura-de-pantalla-2022-10-03-203552.png](https://i.postimg.cc/c1zS9wNm/Captura-de-pantalla-2022-10-03-203552.png)]

> Se puede ver gráficamente como un árbol donde cada raíz es una una solución:

[![Captura-de-pantalla-2022-09-30-093308.png](https://i.postimg.cc/MGK1d1ZD/Captura-de-pantalla-2022-09-30-093308.png)]

Podemos ver que funciona pero **es ineficiente**, ya que el algoritmo llama a sí mismo repetidamente sobre subproblemas que ya ha resuelto.


#**3. Solución con Programación Dinámica (Bottom-up)**

###**3.1 Implementación**

En el código a continuación se implementa la solución con **programación dínamica** para el problema de **Cutting a Rod**. 

Además de el retorno máximo posible, tambien retornar los cortes necesarios la **cantidad de subproblemas** que fue necesario resolver para encontrar la solución óptima.

Tambien se implemanta la función `verbose`, que nos permite ver lo que sucede en al algoritmo paso a paso.

In [14]:
from math import inf
import random

def CutRodPD(p, n, verbose=True):
  r = [0] * (n+1)
  cuts = [0] * (n+1)
  r[0] = 0
  s = 0
  if verbose==True: print("Para una varilla de tamaño", n,"se resuelven solo siguientes subproblemas")
  for j in range(1, n+1):
    q = -inf
    c = 0
    s+= 1
    for i in range(1,j+1):
      s+= 1
      if q < p[i-1] + r[j-i] :
        q = p[i-1] + r[j-i]
        if j-i == 0 :
          c = 0
        else: 
          c = cuts[j-i] + 1
      
    if verbose==True: print("r[",j,"]", "=", q, "fueron necesarios", c, "cortes.")
    cuts[j] = c  
    r[j] = q
  return r[n], cuts[n], s


#ejemplo

p = [1,5,8,9,10,17,17,20,24,30]
print("Tabla de precios:",p)
print()

for i in range(3):
  n = random.randint(1,10)
  m, c , s = CutRodPD(p, n)
  print()
  print("Maxima ganancia para una varilla de longitud", n ,"es", m,",")
  print("fueron necesarios", c,"cortes y",s,"subploblemas fueron resueltos")
  print()

print()



Tabla de precios: [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

Para una varilla de tamaño 1 se resuelven lo siguiente subproblemas
r[ 1 ] = 1 fueron necesarios 0 cortes.

Maxima ganancia para una varilla de longitud 1 es 1 ,
fueron necesarios 0 cortes y 2 subploblemas fueron resueltos

Para una varilla de tamaño 10 se resuelven lo siguiente subproblemas
r[ 1 ] = 1 fueron necesarios 0 cortes.
r[ 2 ] = 5 fueron necesarios 0 cortes.
r[ 3 ] = 8 fueron necesarios 0 cortes.
r[ 4 ] = 10 fueron necesarios 1 cortes.
r[ 5 ] = 13 fueron necesarios 1 cortes.
r[ 6 ] = 17 fueron necesarios 0 cortes.
r[ 7 ] = 18 fueron necesarios 1 cortes.
r[ 8 ] = 22 fueron necesarios 1 cortes.
r[ 9 ] = 25 fueron necesarios 1 cortes.
r[ 10 ] = 30 fueron necesarios 0 cortes.

Maxima ganancia para una varilla de longitud 10 es 30 ,
fueron necesarios 0 cortes y 65 subploblemas fueron resueltos

Para una varilla de tamaño 10 se resuelven lo siguiente subproblemas
r[ 1 ] = 1 fueron necesarios 0 cortes.
r[ 2 ] = 5 fueron necesar

###**3.1 Descripcion del algoritmo**

El problema reside en que en que deben resolver todos los posibles casos, incluso cuando ya fueron resueltos con anterioridad, por no que su tiempo de ejecución es de $O(2^n)$.

Para estos tipos de problemas donde las soluciones se repiten varias veces, la implementar programación dinámica es especialmente eficaz. Además de que se muy usado en problemas de optimización 

> **Resolución por programación dinámica:** Resuelve los problemas solo una vez y guarda el resultado en un arreglo para usarlos cuando sean necesarios nuevamente.

###**3.2 Correctitud del algoritmo**
 
Podemos probar que el algoritmo es correcto usando **inducción**:

>**Caso base:** El valor de una varilla de largo 0 es 0. 

>Y si asumimos que el algoritmo es correcto para los subproblemas mas pequeños $r[j-i]$, tenemos el valor del maximo ingreso para varillas de largo $< j$ y el precio de venta de la primera pieza $p_i$. Y de la mano a la ecuación de subestructura optima probamos que el **algoritmo es correcto**.

 >$r_n=max(p_i+r$<sub>$n−i$</sub>$)$, donde $1≤i≤n$

#**4. Analisis del tiempo de ejecución**
###**4.1 Solucion recursiva**

Sea $T(n) =$ el numero de llamadas recursivas para cualquier $n$.

*   $T(0)=1$
*   $T(n)=1+\sum\limits_{j=0}^{n-1}T(j)$.
 
>**Solución:** $T(n)=2^n$, la complejidad temporal es exponencial con respecto al tamaño del problema.

**Complejidad espacial:**
>$O(1)$, no utiliza espacio extra.

###**4.2 Solucion con Programción Dínamica**

Para el tiempo de ejecucion este algorimo, tenemos un doble loop anidado, y el numero de iteraciones del loop interior forma una progresion aritmetica.

> T(n) = $T(n-1) + T(n-2) +…..+T(1) + 1$.

**Es decir:**

*   $T(1)=1$
*   $T(2) = T(1) + 1= 2$
*   $T(3) = T(2) + T(1)+ 1 = 2 + 1 + 1$
*   $T(n) = n + n-1 + n-2 + …$ es una progresión aritmética

> $T(n)=n+\sum\limits_{i=1}^{n-1}n-i$.

**Solución:** $O(n^2)$


**Tiempo de ejecución de cada subproblema:**
> Cada subproblema se resuelve en $n-i$ comparaciones.

**Complejidad espacial:**
  
> $O(n)$ correspondiente al espacio por el arreglo $r[0...n]$.



#5. Experimentación